# MCQ (Multiple Choice Question) 평가 튜토리얼

## MCQDataset

이 튜토리얼에서는 Huggingface의 객관식 dataset을 불러와서 평가 후 재업로드하는 과정까지 경험해볼 것입니다.

### 1. 데이터셋 불러오기
먼저 HuggingFace Hub에서 데이터셋을 불러오는 방법을 알아보겠습니다:

In [1]:
from langmetrics.llmdataset import LLMDataset
from langmetrics.llmtestcase import LLMTestCase
from datasets import load_dataset
import pandas as pd
from dotenv import load_dotenv

In [2]:
load_dotenv(override=True)

True

In [3]:
dataset = load_dataset('sickgpt/001_MedQA_raw')

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'expected_output', 'choices'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['question', 'expected_output', 'choices'],
        num_rows: 1273
    })
})

이제 LLMDataset을 이용해서 불러와봅시다.

먼저 LLMTestCase는 input, choices, expected_output을 고정으로 받습니다. 그런데 위에 Dataset은 input이 question이라는 열로 되어있네요. field_mapping 인자를 이용해서 column을 매핑해주겠습니다.

In [5]:
LLMTestCase.__annotations__

{'input': str,
 'output': typing.Optional[str],
 'expected_output': str,
 'context': typing.Optional[typing.List[str]],
 'retrieval_context': typing.Optional[typing.List[str]],
 'reasoning': typing.Optional[str],
 'choices': typing.Optional[str]}

In [6]:
# 예시 사용법
field_mapping = {
    'input': 'question',  # 데이터셋의 'question' 필드를 'input'으로 매핑
    'expected_output': 'expected_output',
    'choices': 'choices'
}

In [7]:
dataset = LLMDataset.from_huggingface_hub('sickgpt/001_MedQA_raw', field_mapping=field_mapping)

In [8]:
print(len(dataset))

10178


In [9]:
test_dataset = LLMDataset.from_huggingface_hub('sickgpt/001_MedQA_raw', field_mapping=field_mapping, split='test')

In [10]:
len(test_dataset)

1273

이제 evaluate을 진행해봅시다.

In [11]:
from langmetrics.llmfactory import LLMFactory

In [12]:
LLMFactory.get_model_list()

['gpt-4o',
 'gpt-4o-mini',
 'deepseek-v3',
 'deepseek-reasoner',
 'claude-3.5-sonnet',
 'claude-3.5-haiku',
 'naver']

In [13]:
from langmetrics.llmfactory import LLMFactory
# LLM 모델 생성
gpt_4o_mini = LLMFactory.create_llm('gpt-4o-mini')

In [15]:
from langmetrics.metrics import MCQMetric
metric = MCQMetric(
    answer_model=gpt_4o_mini,
    template_language='en',  # 'ko' 또는 'en'
    generate_template_type='reasoning'  # 'reasoning' 또는 'only_answer'
)

async를 통해서 빠르게 추론을 할 것입니다.

In [17]:
import nest_asyncio
nest_asyncio.apply()

In [18]:
print(test_dataset[0])

LLMTestCase(input='A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?', output=None, expected_output='B', context=None, retrieval_context=None, reasoning=None, choices=['Disclose the error to the patient and put it in the operative report', 'Tell the attending that he cannot fail to disclose this mistake', 'Report the physician to the ethics committee', 'Refuse to dictate the operative report'])


In [19]:
results = await metric.ameasure(test_dataset[:3])

In [20]:
results

LLMDataset(Polars DataFrame with 3 rows)

In [21]:
# r1_results = await r1_metric.ameasure(test_dataset[:10])

약 1200개의 달하는 test를 단 30초만에 모두 추론한 것을 확인할 수 있습니다!

In [22]:
results.df

input,student_answer,teacher_answer,expected_output,context,retrieval_context,reasoning,choices,score,metadata
str,str,null,str,null,null,str,list[str],i64,struct[3]
"""A junior orthopaedic surgery r…","""{ ""reasoning"": ""In this sc…",null,"""B""",null,null,"""In this scenario, the resident…","[""Disclose the error to the patient and put it in the operative report"", ""Tell the attending that he cannot fail to disclose this mistake"", … ""Refuse to dictate the operative report""]",0,"{""en"",""gpt-4o-mini-2024-07-18"",{137,279,416}}"
"""A 67-year-old man with transit…","""{ ""reasoning"": ""In this ca…",null,"""D""",null,null,"""In this case, the patient has …","[""Inhibition of proteasome"", ""Hyperstabilization of microtubules"", … ""Cross-linking of DNA""]",0,"{""en"",""gpt-4o-mini-2024-07-18"",{144,238,382}}"
"""Two weeks after undergoing an …","""{ ""reasoning"": ""The patien…",null,"""B""",null,null,"""The patient's symptoms and lab…","[""Renal papillary necrosis"", ""Cholesterol embolization"", … ""Polyarteritis nodosa""]",1,"{""en"",""gpt-4o-mini-2024-07-18"",{162,407,569}}"


In [23]:
scores = sum([i.score for i in results]) / len(results)

In [24]:
print(scores)

0.3333333333333333


In [27]:
print(results[0])

LLMResult(input='A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?', student_answer='{\n    "reasoning": "In this scenario, the resident faces an ethical dilemma involving disclosure of a medical error. It is essential to maintain transparency in medical practice for patient safety and trust. While the attending physician suggests not to report the complication, the resident has a professional and ethical obligation to disclose the error

In [34]:
results.df['metadata']

metadata
struct[3]
"{""en"",""gpt-4o-mini-2024-07-18"",{137,279,416}}"
"{""en"",""gpt-4o-mini-2024-07-18"",{144,238,382}}"
"{""en"",""gpt-4o-mini-2024-07-18"",{162,407,569}}"
